# **Machine to Machine (M2M) Querying [*MetadataFilters*](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataFilter)**

#### **What is M2M?**
> EarthExplorer (EE) serves as a key data access portal, offering a range of tools for searching, discovering, and downloading data and metadata from the USGS Earth Resources Observation and Science (EROS) data repository. The Machine-to-Machine (M2M) API allows users to search and retrieve download URLS and metadata from the EROS archive by using programming languages like Python or PHP. Users can create JSON structures to pass to M2M endpoints, subsequently receiving JSON responses in return.

#### **Requesting Access**
> Users will need to log on to their [**EROS Registration Service (ERS)**](https://ers.cr.usgs.gov/) accounts to view the M2M documentation. To submit download requests through M2M, ERS users must be authorized. 
> Steps to request M2M access:
> 1. Login to [**EROS Registration Service (ERS)**](https://ers.cr.usgs.gov/) (See [**How to Create an ERS Account**](https://www.youtube.com/watch?v=Ut6kxbuP_nk))
> 2. Select the **Access Request** menu
> 3. From the **Access Controls** page, use the **Request Access** button
> 4. In the **Access Type** selector, choose **Access to EE's Machine to Machine interface (MACHINE)**, then complete the questions


<div class="alert alert-warning"><h4>
The USGS/EROS User Services team assesses these requests. Contact USGS EROS User Services for more information: <a href="https://www.usgs.gov/staff-profiles/usgs-eros-customer-services" target="_blank">custserv@usgs.gov</a> and visit the <a href="https://www.usgs.gov/centers/eros/science/earthexplorer-help-index#ers" target="_blank">EarthExplorer Help Index</a>   </h4></div>


#### **Use Case Scenario**
> This tutorial demonstrates how to query using metadata specific to the various collections of Landsat data. We use the following Machine-to-Machine API filters: [**metadata*Value***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataValue), [**metadata*Between***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataBetween), [**metadata*And***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataAnd), and [**metadata*Or***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataOr).
> <!-- The queryable metadata parameters vary depending on the dataset collection. For instance, Landsat 8-9 OLI/TIRS, Landsat 7 ETM+, and Landsat 4-5 TM may not have the same set of parameters available for querying. -->
>
>
> Users can view queryable metadata after following one of these steps:
> - Submitting a [**dataset-filters**](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters) query and listing the results
> - Submitting a search on [**Earth Explorer**](https://earthexplorer.usgs.gov/) and viewing the list of metadata available for querying under the **Additional Information** tab
>
> The [**Landsat Collection 2 Data Dictionary**](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary) describes all queryable metadata available through Earth Explorer.

---
## **Setup**

#### Import necessary libraries

In [1]:
import json
import requests
from getpass import getpass
import sys
import time
import argparse
import cgi
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

#### Define send request function
This function sends a request to a M2M endpoint and returns the parsed JSON response.

Input parameters include:
- **endpoint_url** (*str*): The URL of the M2M endpoint
- **payload** (*dict*): The payload to be sent with the request

Returns:
- **dict**: Parsed JSON response

In [2]:
# Send http request
def sendRequest(url, data, apiKey = None, exitIfNoResponse = True):
    """
    Send a request to an M2M endpoint and returns the parsed JSON response.

    Parameters:
    endpoint_url (str): The URL of the M2M endpoint
    payload (dict): The payload to be sent with the request

    Returns:
    dict: Parsed JSON response
    """  
    
    json_data = json.dumps(data)
    
    if apiKey == None:
        response = requests.post(url, json_data)
    else:
        headers = {'X-Auth-Token': apiKey}              
        response = requests.post(url, json_data, headers = headers)  
    
    try:
      httpStatusCode = response.status_code 
      if response == None:
          print("No output from service")
          if exitIfNoResponse: sys.exit()
          else: return False
      output = json.loads(response.text)
      if output['errorCode'] != None:
          print(output['errorCode'], "- ", output['errorMessage'])
          if exitIfNoResponse: sys.exit()
          else: return False
      if  httpStatusCode == 404:
          print("404 Not Found")
          if exitIfNoResponse: sys.exit()
          else: return False
      elif httpStatusCode == 401: 
          print("401 Unauthorized")
          if exitIfNoResponse: sys.exit()
          else: return False
      elif httpStatusCode == 400:
          print("Error Code", httpStatusCode)
          if exitIfNoResponse: sys.exit()
          else: return False
    except Exception as e: 
          response.close()
          print(e)
          if exitIfNoResponse: sys.exit()
          else: return False
    response.close()
    
    return output['data']

---
## **ERS User Login**

<div class="alert alert-warning">
    <h3>The USGS M2M Server will no longer support using your ERS <b>passwords</b> for authentication in scripts but instead requires <b>tokens</b>. For more information, visit the <a href="https://www.usgs.gov/media/files/m2m-application-token-documentation" target="_blank">M2M Application Token Documentation</a> or contact <a href="mailto:custserv@usgs.gov" target="_blank">USGS EROS User Services</a>.</h3>
    <h4><p>To get your token, please follow these steps:</p>
    <ol>
        <li>Navigate to <a href="https://ers.cr.usgs.gov/" target="_blank">ERS</a> and log in to your ERS account.</li>
        <li>Click "Create Application Token" under the "Application Tokens" section of the profile page.</li>
        <li>Set the "Token Name" and "Token Expiration."</li>
        <li>Make sure to copy/save your token so it is easily accessible and can be used to download data with the API.</li>
    </ol>
    </h4>
</div>

- ### Login by entering [EROS Registration System (EROS)](https://ers.cr.usgs.gov/https://ers.cr.usgs.gov/) ***`username`*** and ***`token`*** when prompted

In [3]:
def prompt_ERS_login(serviceURL):
    print("Logging in...\n")

    p = ['Enter EROS Registration System (ERS) Username: ', 'Enter ERS Account Token: ']

    # Use requests.post() to make the login request
    response = requests.post(f"{serviceUrl}login-token", json={'username': getpass(prompt=p[0]), 'token': getpass(prompt=p[1])})

    if response.status_code == 200:  # Check for successful response
        apiKey = response.json()['data']
        print('\nLogin Successful, API Key Received!')
        headers = {'X-Auth-Token': apiKey}
        return apiKey
    else:
        print("\nLogin was unsuccessful, please try again or create an account at: https://ers.cr.usgs.gov/register.")

In [4]:
serviceUrl = "https://m2m.cr.usgs.gov/api/api/json/stable/"
apiKey = prompt_ERS_login(serviceUrl)

# Print the API key only if login was successful
# if apiKey:
#     print("\nAPI Key: " + apiKey + "\n"))

Logging in...



Enter EROS Registration System (ERS) Username:  ········
Enter ERS Account Token:  ········



Login Successful, API Key Received!


#### **OR**

- ### Login by setting [EROS Registration System (EROS)](https://ers.cr.usgs.gov/https://ers.cr.usgs.gov/) ***`username`*** and ***`token`***

In [5]:
# username = ""
# token = ""

In [6]:
# print("Logging in...\n")
    
# serviceUrl = "https://m2m.cr.usgs.gov/api/api/json/stable/"
# login_payload = {'username' : username, 'token' : token}
    
# apiKey = sendRequest(serviceUrl + "login-token", login_payload)
    
# print("API Key: " + apiKey + "\n")

---
## **Setup General Search Payload for [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search)**
Below, we've setup a general search payload—a JSON  dictionary with various query parameters—to be transmitted to the M2M [**scene-search**](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) endpoint for querying products within the various collections of Landsat data. This payload includes:

- **maxResults:** the number of results to be returned (default is 100).

- **datasetName:** used to identify the collection of data to search.

- [**sceneFilters:**](https://m2m.cr.usgs.gov/api/docs/datatypes/#sceneFilter) used to filter the data within the collections of Landsat data. The ones used in this tutorial are:

    - The [**acquisitionFilter**](https://m2m.cr.usgs.gov/api/docs/datatypes/#acquisitionFilter) uses the ***'start'*** and ***'end'*** dates in ISO format **yyyy-mm-dd** for the time period of interest. Due to variations in acquisition dates for each Landsat dataset, we won't set a fixed parameter. Instead, we'll wait to fill in the date range as appropriate for each dataset. An example setup is:
    `acquisitionFilter = {'start' : '2023-03-01', 'end' : '2023-04-01'}`

    - The [**cloudCoverFilter**](https://m2m.cr.usgs.gov/api/docs/datatypes/#cloudCoverFilter) is used to filter by percent cloud cover by setting a ***'min'*** and ***'max'*** (**%**).
    
    - The [**metadataFilter**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataFilter), which is the focus of this tutorial, includes the [**metadataValue**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataValue), [**metadataAnd**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataAnd), [**metadataOr**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataOr), and [**metadataBetween**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataBetween). Each of these will be explored in detail in the sections that follow.


### **Create a function that creates general search payload**
In this section, we define a search payload function that generates a JSON dictionary which will be used for running a [**scene-search**](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search). The function takes input parameters such as **datasetName**, [**metadataFilter**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataFilter), and [**acquisitionFilter**](https://m2m.cr.usgs.gov/api/docs/datatypes/#acquisitionFilter). Default parameters like **maxResults** (set to 10) and [**cloudCoverFilter**](https://m2m.cr.usgs.gov/api/docs/datatypes/#cloudCoverFilter) (set to 50%) are also included.

In general, this function simplifies the process of generating a JSON dictionary while applying metadataFilters to various Landsat dataset collections.

In [7]:
def create_search_payload(datasetName, metadataFilter, acquisitionFilter):
    # create search payload from input filters
    search_payload = {
        "maxResults": 10,
        'datasetName' : datasetName,
        'sceneFilter' : {
            'metadataFilter': metadataFilter,
            'acquisitionFilter' : acquisitionFilter,
            'cloudCoverFilter' : {'min' : 0, 'max' : 50},
        }
    }
    return search_payload

<div class="alert alert-info" role="alert">
  <h4>
    View the list of Landsat <a href="./M2M_Landsat_Collections_datasetNames.html" target="_blank">&nbsp;datasetNames</a> available for querying the M2M API.
  </h4>
</div>

# [**1. metadata*Value***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataValue)

---
## [**Searching for a *Satellite***](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#satellite)

To show how to use the [**metadataValue**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataValue) we to search for a specific satellite using the [**Landsat 5 data from the Landsat 4-5 Thematic Mapper Collection 2 Level-1**](https://www.usgs.gov/centers/eros/science/usgs-eros-archive-landsat-archives-landsat-4-5-thematic-mapper-collection-2) (***landsat_tm_c2_l1***) dataset.
We start by sending a [**dataset-filters**](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters) request to retrieve the **`'id'`** for the field we are filtering for, in this case it is **`'Satellite'`**. This **`'id'`** will be used in the metadataFilter parameter **`'filterId'`**. The [**dataset-filters**](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters) request returns the [**metadataFilter**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataFilter) parameters for the specified dataset. These values can be used as additional criteria when submitting search queries for specific scenes. The **'datasetName'** input parameter is necessary. 
#### **1. Run [dataset-filters](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters)**

In [8]:
datasetfilters_tmc2L1 = sendRequest(serviceUrl + "dataset-filters", {'datasetName': 'landsat_tm_c2_l1'}, apiKey)

#### **2. Use Pandas to view the [dataset-filters](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters) result**
Below we show the results from the [**dataset-filters**](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters) query as a table using `pandas`. The **`fieldLabel`** collumn provides insights into the queryable fields available for this specific Landsat dataset. The fields in these results are needed for figuring out which filter options are available, and applying them. While the **`fieldConfig.type`** lists one of three types for each fields:  **`Text`**, **`Range`** or **`Select`**. These act as a guide on the appropriate [**metadataFilter**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataFilter) type to apply. 
  
If the `fieldConfig.type` (also `'fieldConfig':'type'`) is:
- `Select`, then check the `valueList` field for valid values to use in a `'filterType':'value'` metadataFilter ([metadataValue](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataValue))
- `Range`, then use a `'filterType':'between'` metadataFilter and provide `'firstValue'` and `'secondValue'` ([metadataBetween](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataBetween))
- `Text`, then use a `'like'` or `'='` operand in a `'filterType':'value'` metadataFilter ([metadataValue](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataValue))


In [9]:
pd.json_normalize(datasetfilters_tmc2L1)

,id,legacyFieldId,dictionaryLink,fieldLabel,searchSql,fieldConfig.type,fieldConfig.filters,fieldConfig.options.size,fieldConfig.validators,fieldConfig.numElements,...,valueList.4,valueList.DAY,valueList.NIGHT,valueList.NO_PCD,valueList.null,valueList.L1TP,valueList.L1GT,valueList.L1GS,valueList.T1,valueList.T2
0,5e83d0a075cea58a,27718,https://www.usgs.gov/centers/eros/science/land...,Landsat Product Identifier L1,LANDSAT_PRODUCT_ID_L1 like ?,Text,"[{'type': 'StringToUpper', 'options': []}, {'t...",45,[],5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5e83d0a0fbbd1ca1,27737,https://www.usgs.gov/centers/eros/science/land...,Landsat Scene Identifier,ENTITY_ID like ?,Text,"[{'type': 'StringToUpper', 'options': []}, {'t...",25,[],5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5e83d0a017869321,27720,https://www.usgs.gov/centers/eros/science/land...,WRS Path,WRS_PATH between ?,Range,[],NaN,[],1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5e83d0a0f068314e,27721,https://www.usgs.gov/centers/eros/science/land...,WRS Row,WRS_ROW between ?,Range,[],NaN,[],1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5e83d0a072ef8b3f,27728,https://www.usgs.gov/centers/eros/science/land...,Satellite,SATELLITE = ?,Select,"[{'type': 'StringToUpper', 'options': []}]",3,[],1,...,Landsat 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5e83d0a031010d95,27729,https://www.usgs.gov/centers/eros/science/land...,Day/Night Indicator,DAY_NIGHT = ?,Select,[],3,[],1,...,NaN,Day,Night,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5e83d0a0574a972d,27734,https://www.usgs.gov/centers/eros/science/land...,Data Anomaly,"coalesce(DATA_ANOMALY, 'null') = ?",Select,[],3,[],1,...,NaN,NaN,NaN,No PCD,None,NaN,NaN,NaN,NaN,NaN
7,5e83d0a0a0867e5c,27730,https://www.usgs.gov/centers/eros/science/land...,Sun Elevation L1,SUN_ELEVATION_L1 between ?,Range,[],NaN,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,5e83d0a04e6ad4d2,27725,https://www.usgs.gov/centers/eros/science/land...,Data Type L1,"RIGHT(DATA_TYPE_L1, 4) = ?",Select,[],4,[],1,...,NaN,NaN,NaN,NaN,NaN,Level 1TP,Level 1GT,Level 1GS,NaN,NaN
9,5e83d0a09419d091,27724,https://www.usgs.gov/centers/eros/science/land...,Collection Category,COLLECTION_CATEGORY = ?,Select,[],3,[],1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tier 1,Tier 2


#### **3. Identify the index for the filter parameter, `fieldLabel : Satellite`**
In this case it is index # 3. We will use the **`id`** for the **`filterId`** in the [metadataValue](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataValue) dictionary below. 
When the **`'fieldConfig':'type'`** is **`Select`**, users are encouraged to check the **`valueList`** field for valid values to use in a metadataValue filter.

The **`valueList`** shows the querable satellite values for this collection of Landsat data they are **`'All'`** for all satellites,  **`'5'`** for Landsat 5 and **`'4'`** for Landsat 4.

In [10]:
datasetfilters_tmc2L1[3]

{'id': '5e83d0a0f068314e',
 'legacyFieldId': 27721,
 'dictionaryLink': 'https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_row',
 'fieldConfig': {'type': 'Range',
  'filters': [],
  'options': [],
  'validators': [],
  'numElements': 1,
  'displayListId': None},
 'fieldLabel': 'WRS Row',
 'searchSql': 'WRS_ROW between ?'}

#### **4. Fill `metadataFilter` parameter with information for querying Landsat-5 satellites**
- The `'filterType'` is `'value'` since the `'fieldConfig':'type'` is `Select` for the `'fieldLabel': 'Satellite'`
- The `'filterId': datasetfilters_tmc2L1[3]['id']` points to the `'id'` value for querying satellite data within the Landsat 4-5 Thematic Mapper Collection 2 Level-1 (`'landsat_tm_c2_l1'`) dataset.
- The `'value' : '5'` is set to query for Landsat 5 scenes within the Landsat 4-5 Thematic Mapper Collection 2 Level-1 (`'landsat_tm_c2_l1'`) dataset.

In [11]:
satellite_filter = {'filterType': 'value',
                 'filterId': datasetfilters_tmc2L1[3]['id'],
                 'value' : '5'}
satellite_filter

{'filterType': 'value', 'filterId': '5e83d0a0f068314e', 'value': '5'}

#### **5. Create the [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) payload for querying Landsat-5 products from the Landsat 4-5 Thematic Mapper Collection 2 Level-1**

Note that the Landsat 4-5 Thematic Mapper dataset was acquired between March 1984 and January 2013 ([**Landsat Missions Timeline**](https://www.usgs.gov/media/images/landsat-missions-timeline)), therefore the **aquisitionFilter** dates should be within those ranges.

In [12]:
satellite_payload = create_search_payload('landsat_tm_c2_l1', satellite_filter, {'start' : '1994-05-01', 'end' : '1994-05-15'})
satellite_payload

{'maxResults': 10,
 'datasetName': 'landsat_tm_c2_l1',
 'sceneFilter': {'metadataFilter': {'filterType': 'value',
   'filterId': '5e83d0a0f068314e',
   'value': '5'},
  'acquisitionFilter': {'start': '1994-05-01', 'end': '1994-05-15'},
  'cloudCoverFilter': {'min': 0, 'max': 50}}}

#### **6. Run [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search)  and view results** 
The code below runs the [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) endpoint and shows the results using pandas. Note the returned results are all Landsat-5 scenes indicated by the **`'LT05'`** in the **`displayId`** names (see the [**Landsat Product ID**](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary/#landsat_product_id) to learn about the naming conventions).

In [13]:
pd.json_normalize((sendRequest(serviceUrl + "scene-search", satellite_payload, apiKey))['results'])

,browse,cloudCover,entityId,displayId,orderingId,metadata,hasCustomizedMetadata,publishDate,options.bulk,options.download,...,options.secondary,selected.bulk,selected.compare,selected.order,spatialBounds.type,spatialBounds.coordinates,spatialCoverage.type,spatialCoverage.coordinates,temporalCoverage.endDate,temporalCoverage.startDate
0,"[{'id': '5e9e83b68ba2f3d4', 'browseRotationEna...",2,LT50030051994135KIS00,LT05_L1TP_003005_19940515_20200913_02_T1,None,[],None,2022-06-22 19:23:13-05,True,True,...,False,False,False,False,Polygon,"[[[-28.97096, 75.93487], [-28.97096, 78.22042]...",Polygon,"[[[-28.97096, 76.93882], [-23.34467, 75.93487]...",1994-05-15 00:00:00,1994-05-15 00:00:00
1,"[{'id': '5e9e83b68ba2f3d4', 'browseRotationEna...",2,LT51560051994135KIS00,LT05_L1TP_156005_19940515_20200913_02_T2,None,[],None,2022-06-22 19:23:13-05,True,True,...,False,False,False,False,Polygon,"[[[94.65126, 75.9411], [94.65126, 78.22676], [...",Polygon,"[[[94.65126, 76.94552], [100.27865, 75.9411], ...",1994-05-15 00:00:00,1994-05-15 00:00:00
2,"[{'id': '5e9e83b68ba2f3d4', 'browseRotationEna...",25,LT50120051994134KIS00,LT05_L1GS_012005_19940514_20200913_02_T2,None,[],None,2022-06-22 19:23:13-05,True,True,...,False,False,False,False,Polygon,"[[[-42.82555, 75.95104], [-42.82555, 78.23686]...",Polygon,"[[[-42.82555, 76.9562], [-37.19638, 75.95104],...",1994-05-14 00:00:00,1994-05-14 00:00:00
3,"[{'id': '5e9e83b68ba2f3d4', 'browseRotationEna...",33,LT51650051994134KIS00,LT05_L1GS_165005_19940514_20200913_02_T2,None,[],None,2022-06-22 19:23:13-05,True,True,...,False,False,False,False,Polygon,"[[[80.74273, 75.93895], [80.74273, 78.22457], ...",Polygon,"[[[80.74273, 76.9432], [86.36974, 75.93895], [...",1994-05-14 00:00:00,1994-05-14 00:00:00
4,"[{'id': '5e9e83b68ba2f3d4', 'browseRotationEna...",0,LT51580051994133KIS00,LT05_L1GS_158005_19940513_20200913_02_T2,None,[],None,2022-06-22 19:23:13-05,True,True,...,False,False,False,False,Polygon,"[[[91.55776, 75.93595], [91.55776, 78.22152], ...",Polygon,"[[[91.55776, 76.93998], [97.18424, 75.93595], ...",1994-05-13 00:00:00,1994-05-13 00:00:00
5,"[{'id': '5e9e83b68ba2f3d4', 'browseRotationEna...",44,LT50570051994129PAC00,LT05_L1TP_057005_19940509_20200913_02_T2,None,[],None,2022-06-22 19:23:13-05,True,True,...,False,False,False,False,Polygon,"[[[-112.38723, 75.93424], [-112.38723, 78.2197...",Polygon,"[[[-112.38723, 76.93813], [-106.76106, 75.9342...",1994-05-09 00:00:00,1994-05-09 00:00:00
6,"[{'id': '5e9e83b68ba2f3d4', 'browseRotationEna...",47,LT51780051994129KIS00,LT05_L1GS_178005_19940509_20200913_02_T2,None,[],None,2022-06-22 19:23:13-05,True,True,...,False,False,False,False,Polygon,"[[[60.66974, 75.92324], [60.66974, 78.2086], [...",Polygon,"[[[60.66974, 76.92631], [66.29394, 75.92324], ...",1994-05-09 00:00:00,1994-05-09 00:00:00
7,"[{'id': '5e9e83b68ba2f3d4', 'browseRotationEna...",39,LT52260051994129KIS00,LT05_L1GS_226005_19940509_20200913_02_T2,None,[],None,2022-06-22 19:23:13-05,True,True,...,False,False,False,False,Polygon,"[[[-13.46359, 75.92324], [-13.46359, 78.2086],...",Polygon,"[[[-13.46359, 76.92631], [-7.83939, 75.92324],...",1994-05-09 00:00:00,1994-05-09 00:00:00
8,"[{'id': '5e9e83b68ba2f3d4', 'browseRotationEna...",2,LT50020051994128KIS00,LT05_L1TP_002005_19940508_20200913_02_T2,None,[],None,2022-06-22 19:23:13-05,True,True,...,False,False,False,False,Polygon,"[[[-27.38026, 75.92324], [-27.38026, 78.2086],...",Polygon,"[[[-27.38026, 76.92631], [-21.75606, 75.92324]...",1994-05-08 00:00:00,1994-05-08 00:00:00
9,"[{'id': '5e9e83b68ba2f3d4', 'browseRotationEna...",0,LT50180051994128KIS00,LT05_L1GS_018005_19940508_20200913_02_T2,None,[],None,2022-06-22 19:23:13-05,True,True,...,False,False,False,False,Polygon,"[[[-52.09693, 75.92324], [-52.09693, 78.2086],...",Polygon,"[[[-52.09693, 76.92631], [-46.47273, 75.92324]...",1994-05-08 00:00:00,1994-05-08 00:00:00


## [**Landsat Scene Identifier**](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary/#landsat_scene_id)

Here we continue to show the uses of the metadataValue filter by searching for products with a specific [**Landsat Scene Identifier**](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary/#landsat_scene_id) in the same Landsat 4-5 Thematic Mapper Collection 2 Level-1 dataset. The [**Landsat Scene Identifier**](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary/#landsat_scene_id), also known as the **entityID**, is a unique identifier assigned to each Landsat scene. It is part of the metadata associated with Landsat scenes and serves as a specific identifier for a particular observation or image taken by a Landsat satellite. The **entityId** is a string composed of various components that provide information about the Landsat scene, such as the satellite platform, sensor, date of acquisition, and path/row. We will be querying for data within a scene with the **entityId**: **`LT50380322011251PAC02`**.

#### **1. Identify the index for the filter parameter, `fieldLabel : Landsat Scene Identifier`**
In this case it is index # 11. 

In [14]:
datasetfilters_tmc2L1[11]

{'id': '5e83d0a0fe02620e',
 'legacyFieldId': 27733,
 'dictionaryLink': 'https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model',
 'fieldConfig': {'type': 'Range',
  'filters': [],
  'options': [],
  'validators': [],
  'numElements': 1,
  'displayListId': None},
 'fieldLabel': 'Geometric RMSE Model (meters)',
 'searchSql': 'GEOMETRIC_RMSE_MODEL between ?'}

#### **2. Fill `metadataFilter` parameter with information for querying a specific Landsat Scene Identifier**
- The `'filterType'` is `'value'` since the `'fieldConfig':'type'` is `Text` for the `'fieldLabel': 'Landsat Scene Identifier'`
- The `'filterId': datasetfilters_tmc2L1[11]['id']` points to the `'id'` value for querying  Landsat Scene Identifier data within the Landsat 4-5 Thematic Mapper Collection 2 Level-1 (`'landsat_tm_c2_l1'`) dataset.
- The `'value' : 'LT50380322011251PAC02'` is set to query for data within a Landsat 4-5 Thematic Mapper Collection 2 Level-1 (`'landsat_tm_c2_l1'`) scene, where *'LT50380322011251PAC02'* is the entityId for the scene
- The `'operand'` is `'='` to find scenes within the specific Scene Identifier

In [15]:
scene_identifier_filter = {'filterType': 'value',
                 'filterId': datasetfilters_tmc2L1[11]['id'],
                 'value' : 'LT50380322011251PAC02',
                           'operand' : '='}
scene_identifier_filter

{'filterType': 'value',
 'filterId': '5e83d0a0fe02620e',
 'value': 'LT50380322011251PAC02',
 'operand': '='}

#### **3. Create the [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) payload used for finding products within the `LT50380322011251PAC02` scene**

In [16]:
scene_identifier_payload = create_search_payload('landsat_tm_c2_l1', scene_identifier_filter, {'start' : '1994-01-01', 'end' : '2013-01-31'})
scene_identifier_payload

{'maxResults': 10,
 'datasetName': 'landsat_tm_c2_l1',
 'sceneFilter': {'metadataFilter': {'filterType': 'value',
   'filterId': '5e83d0a0fe02620e',
   'value': 'LT50380322011251PAC02',
   'operand': '='},
  'acquisitionFilter': {'start': '1994-01-01', 'end': '2013-01-31'},
  'cloudCoverFilter': {'min': 0, 'max': 50}}}

#### **4. Run [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search)  and view results** 
The result shows a single scene with the Scene Identifier `LT50380322011251PAC02`, this is indicated by the `entityId` names.

In [17]:
pd.json_normalize((sendRequest(serviceUrl + "scene-search", scene_identifier_payload, apiKey))['results'])

""


---
## [**Tile Grid Region**](https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary#tile_grid_region)
Another way to use the [**metadataValue**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataValue) filter is for specifying [**Tile Grid Region**](https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary#tile_grid_region) in the Analysis Ready Data (ARD) dataset. The [**Tile Grid Region**](https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary#tile_grid_region) is also specific to Level-3 Landsat 4-9 Collection 2 (C2) Datasets (e.g. [**Dynamic Surface Water Extent (DWSE)**](https://www.usgs.gov/landsat-missions/landsat-collection-2-level-3-dynamic-surface-water-extent-science-product), [**Burned Area**](https://www.usgs.gov/landsat-missions/landsat-collection-2-level-3-burned-area-science-product), [**Fractional Snow Covered Area**](https://www.usgs.gov/landsat-missions/landsat-collection-2-level-3-fractional-snow-covered-area-science-product), [**Fractional Snow Covered Area (fSCA) Statistics**](https://www.usgs.gov/centers/eros/science/usgs-eros-archive-landsat-collection-2-level-3-fsca-statistics-science-product)) which are processed from [**Landsat Collection 2 (C2) U.S. Analysis Ready Data (ARD)**](https://www.usgs.gov/landsat-missions/landsat-collection-2-us-analysis-ready-data).

Our example below uses the [**Landsat 4-9 Collection 2 (C2) U.S. Analysis Ready Data (ARD) Level-2 Tiles (Albers projection)**](https://www.usgs.gov/landsat-missions/landsat-collection-2-us-analysis-ready-data) (***landsat_ard_tile_c2***) to query for tiles in **Alaska**.

#### **1. Run [dataset-filter](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters) endpoint for *`landsat_ard_tile_c2`* and use pandas to view**

In [18]:
datasetfilters_ardtilec2 = sendRequest(serviceUrl + "dataset-filters", {'datasetName': 'landsat_ard_tile_c2'}, apiKey)
pd.json_normalize(datasetfilters_ardtilec2)

,id,legacyFieldId,dictionaryLink,fieldLabel,searchSql,fieldConfig.type,fieldConfig.filters,fieldConfig.validators,fieldConfig.displayListId,valueList.,...,valueList.90,valueList.100,valueList.LANDSAT_9,valueList.LANDSAT_8,valueList.LANDSAT_7,valueList.LANDSAT_5,valueList.LANDSAT_4,valueList.OLI_TIRS,valueList.ETM,valueList.TM
0,60fabf689767f137,None,https://www.usgs.gov/centers/eros/science/ard-...,Tile Grid Region,TILE_GRID_REGION like ?,Select,[],[],2854,All,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,60fabf683e8957ca,None,https://www.usgs.gov/centers/eros/science/ard-...,Tile Grid Horizontal,TILE_GRID_HORIZONTAL between ?,Range,[],[],,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,60fabf68a4fb8ff9,None,https://www.usgs.gov/centers/eros/science/ard-...,Tile Grid Vertical,TILE_GRID_VERTICAL between ?,Range,[],[],,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,60fabf6880073f49,None,https://www.usgs.gov/centers/eros/science/ard-...,Tile Identifier,TILE_ID like ?,Text,[],[],,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,60fabf686abb6abf,None,https://www.usgs.gov/centers/eros/science/ard-...,Tile Production Date (Ex. YYYY/MM/DD),trunc(DATE_Product_GENERATED) between ?,Range,[],"[{'type': 'Date', 'options': {}}]",,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,60fabf68c6c96869,None,https://www.usgs.gov/centers/eros/science/ard-...,Snow Ice,SNOW_ICE_COVER between ?,Range,[],[],,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,60fabf6852c011be,None,https://www.usgs.gov/centers/eros/science/ard-...,Fill (No Data),FILL <= ?,Select,[],[],2855,All,...,Less than 90%,Less than 100%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,60fabf68dce2badd,None,https://www.usgs.gov/centers/eros/science/ard-...,Spacecraft Identifier,spacecraft_id like ?,Select,[],[],2853,All,...,NaN,NaN,Landsat 9,Landsat 8,Landsat 7,Landsat 5,Landsat 4,NaN,NaN,NaN
8,60fabf6810783d37,None,https://www.usgs.gov/centers/eros/science/ard-...,Sensor Identifier,SENSOR_ID like ?,Select,[],[],2856,All,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OLI_TIRS,ETM,TM


#### **2. Identify the index for the filter parameter, `fieldLabel : Tile Grid Region`**
In this case it is index # 0. The **`valueList`** shows the querable Tile Grid Regions which are `'All'` for all regions,  `'CU'` for conterminous United States, `'AK'` for Alaska and `'HI'` for Hawaii.

In [19]:
datasetfilters_ardtilec2[0]

{'id': '60fabf689767f137',
 'legacyFieldId': None,
 'dictionaryLink': 'https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary#tile_grid_region',
 'fieldConfig': {'type': 'Select',
  'filters': [],
  'options': {},
  'validators': [],
  'displayListId': '2854'},
 'fieldLabel': 'Tile Grid Region',
 'searchSql': 'TILE_GRID_REGION like ?',
 'valueList': {'': 'All', 'CU': 'CU', 'AK': 'AK', 'HI': 'HI'}}

#### **3. Fill `metadataFilter` parameter for querying the Tile Grid Region**
- The `'filterType'` is `'value'` since the `'fieldConfig':'type'` is `Select` for the `fieldLabel : Tile Grid Region`
- The `'filterId': datasetfilters_ardtilec2[0]['id']` points to the `'id'` value for querying a specific Tile Grid Region within the Landsat 4-9 Collection 2 (C2) U.S. Analysis Ready Data (ARD) Level-2 Tiles (`'landsat_ard_tile_c2'`) dataset.
- The `'value' : 'AK'` is set to query for tiles in Alaska

In [20]:
tileregion_filter = {'filterId': datasetfilters_ardtilec2[0]['id'],
                 'filterType': 'value',
                 'value': 'AK'}
tileregion_filter

{'filterId': '60fabf689767f137', 'filterType': 'value', 'value': 'AK'}

#### **4. Create [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) payload for retrieving data from a specific Tile Grid Region**

In [21]:
tile_region_payload = create_search_payload('landsat_ard_tile_c2', tileregion_filter, {'start' : '2009-09-01', 'end' : '2009-09-05'})
tile_region_payload

{'maxResults': 10,
 'datasetName': 'landsat_ard_tile_c2',
 'sceneFilter': {'metadataFilter': {'filterId': '60fabf689767f137',
   'filterType': 'value',
   'value': 'AK'},
  'acquisitionFilter': {'start': '2009-09-01', 'end': '2009-09-05'},
  'cloudCoverFilter': {'min': 0, 'max': 50}}}

#### **5. Run [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search)  and view results** 
The code below runs the [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) endpoint and shows the results using pandas. Note the returned results are all Alaska scenes indicated by the **'AK'** in the **`entityId`** and **`displayId`** names (see the [**Landsat Tile Identifier**](https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary#tile_identifier)).

In [22]:
pd.json_normalize((sendRequest(serviceUrl + "scene-search", tile_region_payload, apiKey))['results'])

,browse,cloudCover,entityId,displayId,orderingId,metadata,hasCustomizedMetadata,publishDate,options.bulk,options.download,...,options.secondary,selected.bulk,selected.compare,selected.order,spatialBounds.type,spatialBounds.coordinates,spatialCoverage.type,spatialCoverage.coordinates,temporalCoverage.endDate,temporalCoverage.startDate
0,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",0,LE07_AK_006008_20090905_20210430_02,LE07_AK_006008_20090905_20210430_02,None,[],None,2021-08-26 01:40:02-05,True,True,...,False,False,False,False,Polygon,"[[[-177.46395, 58.10869], [-177.46395, 59.8112...",Polygon,"[[[-177.46395, 58.10869], [-174.14461, 58.1086...",2009-09-05 00:00:00,2009-09-05 00:00:00
1,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",1,LE07_AK_007008_20090905_20210430_02,LE07_AK_007008_20090905_20210430_02,None,[],None,2021-08-26 01:40:02-05,True,True,...,False,False,False,False,Polygon,"[[[-174.94605, 58.53316], [-174.94605, 60.2024...",Polygon,"[[[-174.94605, 58.53316], [-171.65114, 58.5331...",2009-09-05 00:00:00,2009-09-05 00:00:00
2,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",8,LE07_AK_008007_20090905_20210430_02,LE07_AK_008007_20090905_20210430_02,None,[],None,2021-08-26 01:40:02-05,True,True,...,False,False,False,False,Polygon,"[[[-173.13586, 60.20248], [-173.13586, 61.8451...",Polygon,"[[[-173.13586, 60.20248], [-169.72434, 60.2024...",2009-09-05 00:00:00,2009-09-05 00:00:00
3,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",23,LE07_AK_008008_20090905_20210430_02,LE07_AK_008008_20090905_20210430_02,None,[],None,2021-08-26 01:40:02-05,True,True,...,False,False,False,False,Polygon,"[[[-172.36431, 58.90999], [-172.36431, 60.5429...",Polygon,"[[[-172.36431, 58.90999], [-169.10567, 58.9099...",2009-09-05 00:00:00,2009-09-05 00:00:00
4,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",46,LE07_AK_012002_20090905_20210430_02,LE07_AK_012002_20090905_20210430_02,None,[],None,2021-08-26 01:40:02-05,True,True,...,False,False,False,False,Polygon,"[[[-164.06715, 67.9189], [-164.06715, 69.43082...",Polygon,"[[[-164.06715, 67.9189], [-159.99746, 67.9189]...",2009-09-05 00:00:00,2009-09-05 00:00:00
5,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",0,LE07_AK_012003_20090905_20210430_02,LE07_AK_012003_20090905_20210430_02,None,[],None,2021-08-26 01:40:02-05,True,True,...,False,False,False,False,Polygon,"[[[-163.53154, 66.57972], [-163.53154, 68.0770...",Polygon,"[[[-163.53154, 66.57972], [-159.69194, 66.5797...",2009-09-05 00:00:00,2009-09-05 00:00:00
6,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",0,LE07_AK_016005_20090905_20210430_02,LE07_AK_016005_20090905_20210430_02,None,[],None,2021-08-26 01:55:02-05,True,True,...,False,False,False,False,Polygon,"[[[-149.92786, 63.96527], [-149.92786, 65.4207...",Polygon,"[[[-149.92786, 63.96527], [-146.52133, 63.9652...",2009-09-05 00:00:00,2009-09-05 00:00:00
7,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",2,LE07_AK_016006_20090905_20210430_02,LE07_AK_016006_20090905_20210430_02,None,[],None,2021-08-26 01:55:02-05,True,True,...,False,False,False,False,Polygon,"[[[-150.10817, 62.63013], [-150.10817, 64.0785...",Polygon,"[[[-150.10817, 62.63013], [-146.86594, 62.6301...",2009-09-05 00:00:00,2009-09-05 00:00:00
8,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",34,LE07_AK_016007_20090905_20210430_02,LE07_AK_016007_20090905_20210430_02,None,[],None,2021-08-26 01:55:02-05,True,True,...,False,False,False,False,Polygon,"[[[-150.27321, 61.29548], [-150.27321, 62.7382...",Polygon,"[[[-150.27321, 61.29548], [-147.18029, 61.2954...",2009-09-05 00:00:00,2009-09-05 00:00:00
9,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",8,LE07_AK_016008_20090905_20210430_02,LE07_AK_016008_20090905_20210430_02,None,[],None,2021-08-26 01:55:02-05,True,True,...,False,False,False,False,Polygon,"[[[-150.42483, 59.96055], [-150.42483, 61.3989...",Polygon,"[[[-150.42483, 59.96055], [-147.4682, 59.96055...",2009-09-05 00:00:00,2009-09-05 00:00:00


<div class="alert alert-info"><h4>To understand the tile naming convention, please refer to the <a href="https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary#tile_identifier" target="_blank"><b><i>Tile Identifier </i></b></a>documentation.</h4></div>

# [**metadata*Between***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataBetween)

The [**metadataBetween**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataBetween) is used to find products within a specified **`Range`**, i.e. values or dates, depending on the metadata being queried. A reminder that if the **`fieldConfig.tile`** is **`Range`**, a  **`firstValue`** and **`secondValue`** are necessary.

---
## [**Tile Grid Horizontal**](https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary#tile_grid_horizontal)
Here we use the [**metadataBetween**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataBetween) filter to find products within a range of [**Tile Grid Horizontal**](https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary#tile_grid_horizontal) with the same [**Landsat 4-9 Collection 2 (C2) U.S. Analysis Ready Data (ARD) Level-2 Tiles (Albers projection)**](https://www.usgs.gov/centers/eros/science/usgs-eros-archive-landsat-archives-landsat-collection-2-us-analysis-ready-data) (***landsat_ard_tile_c2***) dataset used above.

#### **1. Identify the index for the filter parameter, `fieldLabel : Tile Grid Horizontal`**
In this case it is index # 1. We will use the **`id`** for the **`filterId`** in the metadataBetween dictionary below. 

In [23]:
datasetfilters_ardtilec2[1]

{'id': '60fabf683e8957ca',
 'legacyFieldId': None,
 'dictionaryLink': 'https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary#tile_grid_horizontal',
 'fieldConfig': {'type': 'Range',
  'filters': [],
  'options': {},
  'validators': [],
  'displayListId': ''},
 'fieldLabel': 'Tile Grid Horizontal',
 'searchSql': 'TILE_GRID_HORIZONTAL between ?'}

#### **2. Fill `metadataFilter` parameter**
- The `'filterType'` is `'between'` since the `'fieldConfig':'type'` is `Range` for the `'fieldLabel': 'Tile Grid Horizontal'`
- The `'filterId': datasetfilters_ardtilec2[1]['id']` points to the `'id'` value for querying Tile Grid Horizontal ranges within this particular dataset
- The `'firstValue' : '8'` is the start of the range and `'secondValue': 10` is the end of the range

In [24]:
tilehorizontal_filter = {'filterType': 'between',
                    'filterId': datasetfilters_ardtilec2[1]['id'],
                    'firstValue': 8,
                    'secondValue': 10}
tilehorizontal_filter

{'filterType': 'between',
 'filterId': '60fabf683e8957ca',
 'firstValue': 8,
 'secondValue': 10}

#### **3. Create and view search payload for the Tile Grid Horizontal payload**

In [25]:
tile_horizontal_payload = create_search_payload('landsat_ard_tile_c2', tilehorizontal_filter, {'start' : '2023-01-01', 'end' : '2023-11-01'})
tile_horizontal_payload

{'maxResults': 10,
 'datasetName': 'landsat_ard_tile_c2',
 'sceneFilter': {'metadataFilter': {'filterType': 'between',
   'filterId': '60fabf683e8957ca',
   'firstValue': 8,
   'secondValue': 10},
  'acquisitionFilter': {'start': '2023-01-01', 'end': '2023-11-01'},
  'cloudCoverFilter': {'min': 0, 'max': 50}}}

#### **4. Run [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search)  and view results**
The results show the **HHH** (Horizontal Tile Number) in both the **`entityIds`** and **`displayIds`** ranging from **`'008'`** to **`'010'`**, for an ARD with a naming convention of **`LXSS_US_HHHVVV_YYYYMMDD_yyyymmdd_CCC_VVV`** (see the ARD naming conventions in the [**Tile Identifier**](https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary/#tile_identifier)).

In [26]:
pd.json_normalize((sendRequest(serviceUrl + "scene-search", tile_horizontal_payload, apiKey))['results'])

,browse,cloudCover,entityId,displayId,orderingId,metadata,hasCustomizedMetadata,publishDate,options.bulk,options.download,...,options.secondary,selected.bulk,selected.compare,selected.order,spatialBounds.type,spatialBounds.coordinates,spatialCoverage.type,spatialCoverage.coordinates,temporalCoverage.endDate,temporalCoverage.startDate
0,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",21,LC08_CU_008001_20231101_20231116_02,LC08_CU_008001_20231101_20231116_02,None,[],None,2023-11-16 11:25:02-06,True,True,...,False,False,False,False,Polygon,"[[[-114.93143, 48.93643], [-114.93143, 50.5369...",Polygon,"[[[-114.93143, 48.93643], [-112.53892, 48.9364...",2023-11-01 00:00:00,2023-11-01 00:00:00
1,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",42,LC08_CU_008002_20231101_20231116_02,LC08_CU_008002_20231101_20231116_02,None,[],None,2023-11-16 11:25:02-06,True,True,...,False,False,False,False,Polygon,"[[[-114.53116, 47.60003], [-114.53116, 49.1870...",Polygon,"[[[-114.53116, 47.60003], [-112.19456, 47.6000...",2023-11-01 00:00:00,2023-11-01 00:00:00
2,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",9,LC08_CU_008003_20231101_20231116_02,LC08_CU_008003_20231101_20231116_02,None,[],None,2023-11-16 11:25:02-06,True,True,...,False,False,False,False,Polygon,"[[[-114.14726, 46.2689], [-114.14726, 47.84423...",Polygon,"[[[-114.14726, 46.2689], [-111.86412, 46.2689]...",2023-11-01 00:00:00,2023-11-01 00:00:00
3,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",1,LC08_CU_009001_20231101_20231116_02,LC08_CU_009001_20231101_20231116_02,None,[],None,2023-11-16 11:25:02-06,True,True,...,False,False,False,False,Polygon,"[[[-112.89808, 49.18707], [-112.89808, 50.7657...",Polygon,"[[[-112.89808, 49.18707], [-110.53189, 49.1870...",2023-11-01 00:00:00,2023-11-01 00:00:00
4,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",30,LC08_CU_009002_20231101_20231116_02,LC08_CU_009002_20231101_20231116_02,None,[],None,2023-11-16 11:25:02-06,True,True,...,False,False,False,False,Polygon,"[[[-112.53892, 47.84423], [-112.53892, 49.4096...",Polygon,"[[[-112.53892, 47.84423], [-110.22796, 47.8442...",2023-11-01 00:00:00,2023-11-01 00:00:00
5,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",0,LC08_CU_008014_20231031_20231116_02,LC08_CU_008014_20231031_20231116_02,None,[],None,2023-11-16 10:55:03-06,True,True,...,False,False,False,False,Polygon,"[[[-110.77244, 31.71677], [-110.77244, 33.2431...",Polygon,"[[[-110.77244, 31.71677], [-108.95162, 31.7167...",2023-10-31 00:00:00,2023-10-31 00:00:00
6,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",0,LC08_CU_009011_20231031_20231116_02,LC08_CU_009011_20231031_20231116_02,None,[],None,2023-11-16 11:40:02-06,True,True,...,False,False,False,False,Polygon,"[[[-109.87907, 35.89921], [-109.87907, 37.4072...",Polygon,"[[[-109.87907, 35.89921], [-107.97149, 35.8992...",2023-10-31 00:00:00,2023-10-31 00:00:00
7,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",0,LC08_CU_009012_20231031_20231116_02,LC08_CU_009012_20231031_20231116_02,None,[],None,2023-11-16 11:40:02-06,True,True,...,False,False,False,False,Polygon,"[[[-109.63506, 34.57237], [-109.63506, 36.0791...",Polygon,"[[[-109.63506, 34.57237], [-107.76397, 34.5723...",2023-10-31 00:00:00,2023-10-31 00:00:00
8,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",0,LC08_CU_009013_20231031_20231116_02,LC08_CU_009013_20231031_20231116_02,None,[],None,2023-11-16 10:55:03-06,True,True,...,False,False,False,False,Polygon,"[[[-109.39942, 33.2431], [-109.39942, 34.74941...",Polygon,"[[[-109.39942, 33.2431], [-107.56348, 33.2431]...",2023-10-31 00:00:00,2023-10-31 00:00:00
9,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",0,LC08_CU_009014_20231031_20231116_02,LC08_CU_009014_20231031_20231116_02,None,[],None,2023-11-16 10:55:03-06,True,True,...,False,False,False,False,Polygon,"[[[-109.17174, 31.9108], [-109.17174, 33.41744...",Polygon,"[[[-109.17174, 31.9108], [-107.36968, 31.9108]...",2023-10-31 00:00:00,2023-10-31 00:00:00


---
## [**Tile Production Date**](https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary#tile_production_date)
We can also filter using the production date **`Tile Production Date (Ex. YYYY/MM/DD)`** metadataFilter using the [**Landsat 4-9 Collection 2 (C2) U.S. Analysis Ready Data (ARD) Level-2 Tiles (Albers projection)**](https://www.usgs.gov/landsat-missions/landsat-collection-2-us-analysis-ready-data) (***landsat_ard_tile_c2***). Here, we limit the search to production dates between **July-1-2023** and **July-31-2023**.

#### **1. Identify the index for the filter parameter, `fieldLabel : Tile Production Date (Ex. YYYY/MM/DD)`**
In this case it is index # 4. We will use the **`id`** for the **`filterId`** in the metadataBetween dictionary below. 

In [27]:
datasetfilters_ardtilec2[4]

{'id': '60fabf686abb6abf',
 'legacyFieldId': None,
 'dictionaryLink': 'https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary#tile_production_date',
 'fieldConfig': {'type': 'Range',
  'filters': [],
  'options': {},
  'validators': [{'type': 'Date', 'options': {}}],
  'displayListId': ''},
 'fieldLabel': 'Tile Production Date (Ex. YYYY/MM/DD)',
 'searchSql': 'trunc(DATE_Product_GENERATED) between ?'}

#### **2. Fill Tile Production Date `metadataFilter` parameter**
- The `'filterType'` is `'between'` since the `'fieldConfig':'type'` is `Range` for the `'fieldLabel': 'Tile Production Date (Ex. YYYY/MM/DD)'`
- The `'filterId': datasetfilters_ardtilec2[4]['id']` points to the `'id'` value for querying Tile Production Date ranges within this particular dataset
- The `'firstValue' : '2023/07/01'` is the start of the date range and `'secondValue': 2023/07/31` is the end of the date range

In [28]:
tileproddate_filter = {'filterId': datasetfilters_ardtilec2[4]['id'],
                   'filterType': 'between',
                   'firstValue': '2023/07/01',
                   'secondValue': '2023/07/31'}
tileproddate_filter

{'filterId': '60fabf686abb6abf',
 'filterType': 'between',
 'firstValue': '2023/07/01',
 'secondValue': '2023/07/31'}

#### **3. Create and View Tile Production Date Search Payload**

Using the similar acquisitionFilter dates as above `{'start' : '2023-01-01', 'end' : '2023-11-01'}` we expect results to be narrowed to the Tile Production Date dates in the new `tileproddate_betweenfilter` paramater i.e. between `'2023/07/01'` and `'2023/07/31'`.

In [29]:
tile_prod_date_payload = create_search_payload('landsat_ard_tile_c2', tileproddate_filter, {'start' : '2023-01-01', 'end' : '2023-11-01'})
tile_prod_date_payload

{'maxResults': 10,
 'datasetName': 'landsat_ard_tile_c2',
 'sceneFilter': {'metadataFilter': {'filterId': '60fabf686abb6abf',
   'filterType': 'between',
   'firstValue': '2023/07/01',
   'secondValue': '2023/07/31'},
  'acquisitionFilter': {'start': '2023-01-01', 'end': '2023-11-01'},
  'cloudCoverFilter': {'min': 0, 'max': 50}}}

#### **4. Run and view the [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) payload results for the Tile Production Date Search**

The results from the query show products within the Tile Production Date where the **`yyyymmdd`** in the [**ARD Tile Identifier**](https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary#tile_identifier) **`LXSS_US_HHHVVV_YYYYMMDD_yyyymmdd_CCC_VVV`** is between the specified production dates **`'2023/07/20'`** and **`'2023/07/31'`**.

In [30]:
pd.json_normalize((sendRequest(serviceUrl + "scene-search", tile_prod_date_payload, apiKey))['results'])

,browse,cloudCover,entityId,displayId,orderingId,metadata,hasCustomizedMetadata,publishDate,options.bulk,options.download,...,options.secondary,selected.bulk,selected.compare,selected.order,spatialBounds.type,spatialBounds.coordinates,spatialCoverage.type,spatialCoverage.coordinates,temporalCoverage.endDate,temporalCoverage.startDate
0,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",47,LC09_AK_010013_20230718_20230723_02,LC09_AK_010013_20230718_20230723_02,None,[],None,2023-07-23 16:55:03-05,True,True,...,False,False,False,False,Polygon,"[[[-164.79294, 52.88518], [-164.79294, 54.4140...",Polygon,"[[[-164.79294, 52.88518], [-162.22905, 52.8851...",2023-07-18 00:00:00,2023-07-18 00:00:00
1,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",17,LC09_AK_011008_20230718_20230723_02,LC09_AK_011008_20230718_20230723_02,None,[],None,2023-07-23 17:40:03-05,True,True,...,False,False,False,False,Polygon,"[[[-164.29714, 59.73981], [-164.29714, 61.2450...",Polygon,"[[[-164.29714, 59.73981], [-161.22152, 59.7398...",2023-07-18 00:00:00,2023-07-18 00:00:00
2,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",40,LC09_AK_011009_20230718_20230723_02,LC09_AK_011009_20230718_20230723_02,None,[],None,2023-07-23 17:40:03-05,True,True,...,False,False,False,False,Polygon,"[[[-163.88369, 58.41228], [-163.88369, 59.9121...",Polygon,"[[[-163.88369, 58.41228], [-160.94077, 58.4122...",2023-07-18 00:00:00,2023-07-18 00:00:00
3,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",40,LC09_AK_011012_20230718_20230723_02,LC09_AK_011012_20230718_20230723_02,None,[],None,2023-07-23 16:55:03-05,True,True,...,False,False,False,False,Polygon,"[[[-162.82007, 54.41402], [-162.82007, 55.9041...",Polygon,"[[[-162.82007, 54.41402], [-160.21547, 54.4140...",2023-07-18 00:00:00,2023-07-18 00:00:00
4,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",0,LC09_AK_011013_20230718_20230723_02,LC09_AK_011013_20230718_20230723_02,None,[],None,2023-07-23 16:55:03-05,True,True,...,False,False,False,False,Polygon,"[[[-162.51437, 53.07455], [-162.51437, 54.5632...",Polygon,"[[[-162.51437, 53.07455], [-160.00616, 53.0745...",2023-07-18 00:00:00,2023-07-18 00:00:00
5,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",0,LC09_AK_012005_20230718_20230723_02,LC09_AK_012005_20230718_20230723_02,None,[],None,2023-07-23 17:25:03-05,True,True,...,False,False,False,False,Polygon,"[[[-162.61398, 63.91008], [-162.61398, 65.3858...",Polygon,"[[[-162.61398, 63.91008], [-159.16546, 63.9100...",2023-07-18 00:00:00,2023-07-18 00:00:00
6,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",1,LC09_AK_012006_20230718_20230723_02,LC09_AK_012006_20230718_20230723_02,None,[],None,2023-07-23 17:25:03-05,True,True,...,False,False,False,False,Polygon,"[[[-162.21809, 62.57742], [-162.21809, 64.0452...",Polygon,"[[[-162.21809, 62.57742], [-158.93707, 62.5774...",2023-07-18 00:00:00,2023-07-18 00:00:00
7,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",30,LC09_AK_012007_20230718_20230723_02,LC09_AK_012007_20230718_20230723_02,None,[],None,2023-07-23 17:25:03-05,True,True,...,False,False,False,False,Polygon,"[[[-161.85682, 61.24501], [-161.85682, 62.7065...",Polygon,"[[[-161.85682, 61.24501], [-158.72799, 61.2450...",2023-07-18 00:00:00,2023-07-18 00:00:00
8,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",35,LC09_AK_012008_20230718_20230723_02,LC09_AK_012008_20230718_20230723_02,None,[],None,2023-07-23 17:40:03-05,True,True,...,False,False,False,False,Polygon,"[[[-161.52584, 59.91211], [-161.52584, 61.3686...",Polygon,"[[[-161.52584, 59.91211], [-158.53587, 59.9121...",2023-07-18 00:00:00,2023-07-18 00:00:00
9,"[{'id': '60fac014d0e80a74', 'browseRotationEna...",25,LC09_AK_012009_20230718_20230723_02,LC09_AK_012009_20230718_20230723_02,None,[],None,2023-07-23 17:40:03-05,True,True,...,False,False,False,False,Polygon,"[[[-161.22152, 58.57805], [-161.22152, 60.0307...",Polygon,"[[[-161.22152, 58.57805], [-158.35873, 58.5780...",2023-07-18 00:00:00,2023-07-18 00:00:00


---
# [**metadata*And***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataAnd)
The [**metadataAnd**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataAnd) is used to make combinations of the [**metadataValue**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataValue) and/or [**metadataBetween**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataBetween)
 filter types.
 For this example we query within the **Landsat 8-9 Operational Land Imager and Thermal Infrared Sensor Collection 2 Level-2 (*landsat_ot_c2_l2*)** dataset.

In [31]:
datasetfilters_otc2l2 = sendRequest(serviceUrl + "dataset-filters", {'datasetName': 'landsat_ot_c2_l2'}, apiKey)
pd.json_normalize(datasetfilters_otc2l2)

,id,legacyFieldId,dictionaryLink,fieldLabel,searchSql,fieldConfig.type,fieldConfig.filters,fieldConfig.options.size,fieldConfig.validators,fieldConfig.numElements,...,valueList.8,fieldConfig.options.multiple,valueList.OLI,valueList.OLI_TIRS,valueList.TIRS,valueList.NADIR,valueList.OFFNADIR,valueList.T1,valueList.T2,valueList.RT
0,5e83d14f567d0086,27819.0,https://www.usgs.gov/centers/eros/science/land...,Landsat Product Identifier L2,EE_DISPLAY_ID like ?,Text,"[{'type': 'StringToUpper', 'options': []}, {'t...",50,[],5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5e83d14f227b0d85,27863.0,https://www.usgs.gov/centers/eros/science/land...,Landsat Product Identifier L1,LANDSAT_PRODUCT_ID_L1 like ?,Text,"[{'type': 'StringToUpper', 'options': []}, {'t...",45,[],5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5e83d14fc84c9a78,27836.0,https://www.usgs.gov/centers/eros/science/land...,Landsat Scene Identifier,ENTITY_ID like ?,Text,"[{'type': 'StringToUpper', 'options': []}, {'t...",30,[],5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5e83d14fb9436d88,27820.0,https://www.usgs.gov/centers/eros/science/land...,WRS Path,WRS_PATH between ?,Range,[],NaN,[],1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5e83d14ff1eda1b8,27821.0,https://www.usgs.gov/centers/eros/science/land...,WRS Row,WRS_ROW between ?,Range,[],NaN,[],1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,61af9273566bb9a8,NaN,https://www.usgs.gov/centers/eros/science/land...,Satellite,satellite = ?,Select,[],NaN,[],NaN,...,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5e83d14fc6e09eb6,27827.0,https://www.usgs.gov/centers/eros/science/land...,Sensor Identifier,SENSOR_ID = ?,Select,"[{'type': 'StringToUpper', 'options': []}]",4,[],1,...,NaN,True,OLI,OLI_TIRS,TIRS,NaN,NaN,NaN,NaN,NaN
7,5f6aa429b870fd7f,NaN,https://www.usgs.gov/centers/eros/science/land...,Nadir/Off Nadir,NADIR_OFFNADIR = ?,Select,"[{'type': 'StringToUpper', 'options': {}}]",NaN,[],NaN,...,NaN,NaN,NaN,NaN,NaN,Nadir,Off Nadir,NaN,NaN,NaN
8,5f6a6fb2137a3c00,NaN,https://www.usgs.gov/centers/eros/science/land...,Collection Category,COLLECTION_CATEGORY = ?,Select,[],NaN,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tier 1,Tier 2,Real-Time
9,5f6a6f82795f1b6c,NaN,https://www.usgs.gov/centers/eros/science/land...,Date Product Generated L2 (YYYY/MM/DD),trunc(DATE_L2_GENERATED) between ?,Range,[],NaN,"[{'type': 'Date', 'options': {}}]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---
## **Combine [**metadata*Between***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataBetween) filters with [metadata*And*](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataOr)**

We can start by finding data within specific ranges of the **`WRS Path`** and **`WRS Row`**.

#### **1. Create and print the [WRS Path](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_row) *between* filter**
The WRS Path is at index # 3.

In [32]:
datasetfilters_otc2l2[3]

{'id': '5e83d14fb9436d88',
 'legacyFieldId': 27820,
 'dictionaryLink': 'https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_path',
 'fieldConfig': {'type': 'Range',
  'filters': [],
  'options': [],
  'validators': [],
  'numElements': 1,
  'displayListId': None},
 'fieldLabel': 'WRS Path',
 'searchSql': 'WRS_PATH between ?'}

- The `'filterType'` is `'between'` since the `'fieldConfig':'type'` is `Range` for the `'fieldLabel': 'WRS Path'`
- The `'filterId': datasetfilters_otc2l2[3]['id']` points to the `'id'` value for querying WRS Path ranges within this particular dataset
- The `'firstValue' : 29` is the start and `'secondValue': 31` is the end of the WRS path range

In [33]:
WRSpath_filter = {'filterId': datasetfilters_otc2l2[3]['id'],
                   'filterType': 'between',
                   'firstValue': 29,
                   'secondValue': 31}
WRSpath_filter

{'filterId': '5e83d14fb9436d88',
 'filterType': 'between',
 'firstValue': 29,
 'secondValue': 31}

#### **2. Create and print the [WRS Row](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_row) *between* filter**
The WRS Path is at index # 4.

In [34]:
datasetfilters_otc2l2[4]

{'id': '5e83d14ff1eda1b8',
 'legacyFieldId': 27821,
 'dictionaryLink': 'https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_row',
 'fieldConfig': {'type': 'Range',
  'filters': [],
  'options': [],
  'validators': [],
  'numElements': 1,
  'displayListId': None},
 'fieldLabel': 'WRS Row',
 'searchSql': 'WRS_ROW between ?'}

- The `'filterType'` is `'between'` since the `'fieldConfig':'type'` is `Range` for the `'fieldLabel': 'WRS Row'`
- The `'filterId': datasetfilters_otc2l2[4]['id']` points to the `'id'` value for querying WRS Row ranges within this particular dataset
- The `'firstValue' : 29` is the start and `'secondValue': 31` is the end of the WRS Row range

In [35]:
WRSrow_filter = {'filterId': datasetfilters_otc2l2[4]['id'],
                   'filterType': 'between',
                   'firstValue': 29,
                   'secondValue': 31}
WRSrow_filter

{'filterId': '5e83d14ff1eda1b8',
 'filterType': 'between',
 'firstValue': 29,
 'secondValue': 31}

#### **3. Combine into a [**metadataAnd**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataAnd) filter parameter**
- The `'filterType'` is `'and'` since we are combining two `'between'` metadataFilters
- The `'childFilters'` are  the `WRSpath_filter` and the `WRSrow_filter`, `'between'` filters created above

In [36]:
WRS_path_row_andfilter = {'filterType': 'and',
              'childFilters':[WRSpath_filter, WRSrow_filter]}

WRS_path_row_andfilter

{'filterType': 'and',
 'childFilters': [{'filterId': '5e83d14fb9436d88',
   'filterType': 'between',
   'firstValue': 29,
   'secondValue': 31},
  {'filterId': '5e83d14ff1eda1b8',
   'filterType': 'between',
   'firstValue': 29,
   'secondValue': 31}]}

#### **4. Create and View WRS Path and Row Search Payload**
We expect results to show dataset between PATHs and ROWs Grid `29` and `31`.

In [37]:
WRS_path_row_payload = create_search_payload('landsat_ot_c2_l2', WRS_path_row_andfilter, {'start' : '2023-01-01', 'end' : '2023-11-01'})
WRS_path_row_payload

{'maxResults': 10,
 'datasetName': 'landsat_ot_c2_l2',
 'sceneFilter': {'metadataFilter': {'filterType': 'and',
   'childFilters': [{'filterId': '5e83d14fb9436d88',
     'filterType': 'between',
     'firstValue': 29,
     'secondValue': 31},
    {'filterId': '5e83d14ff1eda1b8',
     'filterType': 'between',
     'firstValue': 29,
     'secondValue': 31}]},
  'acquisitionFilter': {'start': '2023-01-01', 'end': '2023-11-01'},
  'cloudCoverFilter': {'min': 0, 'max': 50}}}

#### **5. Run and view the [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) payload results for WRS Path and Row Search**

The results from the query show **`displayIDs`** with **`PPP`** (WRS Path number) and **`RRR`** (WRS ROw numbers) from **`29`** to **`31`** (see the [**Landsat Collection 2 Product Identifier**](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary/#landsat_product_id) **`LXSS_LLLL_PPPRRR_YYYYMMDD_yyyymmdd_CC_TX`**). 

In [38]:
pd.json_normalize((sendRequest(serviceUrl + "scene-search", WRS_path_row_payload, apiKey))['results'])

,browse,cloudCover,entityId,displayId,orderingId,metadata,hasCustomizedMetadata,publishDate,options.bulk,options.download,...,options.secondary,selected.bulk,selected.compare,selected.order,spatialBounds.type,spatialBounds.coordinates,spatialCoverage.type,spatialCoverage.coordinates,temporalCoverage.endDate,temporalCoverage.startDate
0,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",49,LC80290302023301LGN00,LC08_L2SP_029030_20231028_20231101_02_T2,None,[],None,2023-11-01 03:33:35-05,True,True,...,False,False,False,False,Polygon,"[[[-98.54497, 42.11196], [-98.54497, 44.24219]...",Polygon,"[[[-98.54497, 42.52909], [-96.29481, 42.11196]...",2023-10-28 00:00:00,2023-10-28 00:00:00
1,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",17,LC80290312023301LGN00,LC08_L2SP_029031_20231028_20231101_02_T2,None,[],None,2023-11-01 03:33:22-05,True,True,...,False,False,False,False,Polygon,"[[[-98.98939, 40.68865], [-98.98939, 42.81543]...",Polygon,"[[[-98.98939, 41.10047], [-96.78753, 40.68865]...",2023-10-28 00:00:00,2023-10-28 00:00:00
2,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",36,LC90300292023300LGN00,LC09_L2SP_030029_20231027_20231028_02_T1,None,[],None,2023-10-28 07:00:55-05,True,True,...,False,False,False,False,Polygon,"[[[-99.63417, 43.53429], [-99.63417, 45.66581]...",Polygon,"[[[-99.63417, 43.95475], [-97.33197, 43.53429]...",2023-10-27 00:00:00,2023-10-27 00:00:00
3,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",6,LC90300302023300LGN00,LC09_L2SP_030030_20231027_20231028_02_T1,None,[],None,2023-10-28 06:59:59-05,True,True,...,False,False,False,False,Polygon,"[[[-100.09364, 42.11325], [-100.09364, 44.2408...",Polygon,"[[[-100.09364, 42.52787], [-97.84337, 42.11325...",2023-10-27 00:00:00,2023-10-27 00:00:00
4,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",1,LC90300312023300LGN00,LC09_L2SP_030031_20231027_20231028_02_T1,None,[],None,2023-10-28 06:58:59-05,True,True,...,False,False,False,False,Polygon,"[[[-100.53788, 40.68971], [-100.53788, 42.8138...",Polygon,"[[[-100.53788, 41.09899], [-98.33614, 40.68971...",2023-10-27 00:00:00,2023-10-27 00:00:00
5,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",6,LC90290292023293LGN00,LC09_L2SP_029029_20231020_20231023_02_T1,None,[],None,2023-10-23 17:19:14-05,True,True,...,False,False,False,False,Polygon,"[[[-98.10604, 43.53455], [-98.10604, 45.66596]...",Polygon,"[[[-98.10604, 43.95502], [-95.80358, 43.53455]...",2023-10-20 00:00:00,2023-10-20 00:00:00
6,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",0,LC90290302023293LGN00,LC09_L2SP_029030_20231020_20231023_02_T1,None,[],None,2023-10-23 17:18:55-05,True,True,...,False,False,False,False,Polygon,"[[[-98.56553, 42.11324], [-98.56553, 44.24081]...",Polygon,"[[[-98.56553, 42.52788], [-96.31495, 42.11324]...",2023-10-20 00:00:00,2023-10-20 00:00:00
7,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",0,LC90290312023293LGN00,LC09_L2SP_029031_20231020_20231023_02_T1,None,[],None,2023-10-23 17:20:46-05,True,True,...,False,False,False,False,Polygon,"[[[-99.00974, 40.68995], [-99.00974, 42.81405]...",Polygon,"[[[-99.00974, 41.09929], [-96.80751, 40.68995]...",2023-10-20 00:00:00,2023-10-20 00:00:00
8,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",1,LC80300292023292LGN00,LC08_L2SP_030029_20231019_20231103_02_T1,None,[],None,2023-11-03 01:25:30-05,True,True,...,False,False,False,False,Polygon,"[[[-99.61313, 43.5334], [-99.61313, 45.66746],...",Polygon,"[[[-99.61313, 43.95627], [-97.31153, 43.5334],...",2023-10-19 00:00:00,2023-10-19 00:00:00
9,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",0,LC80300302023292LGN00,LC08_L2SP_030030_20231019_20231103_02_T1,None,[],None,2023-11-03 01:25:57-05,True,True,...,False,False,False,False,Polygon,"[[[-100.07273, 42.11201], [-100.07273, 44.2422...",Polygon,"[[[-100.07273, 42.52905], [-97.82303, 42.11201...",2023-10-19 00:00:00,2023-10-19 00:00:00


---
## **Combine a [**metadata*Value***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataValue) and [**metadata*Between***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataBetween) filter with [metadata*And*](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataOr)**

We can combine both **`WRSpath_filter`** and **`WRSrow_filter`** **`'between'`** filters created above with a **Collection Category** **`'value'`** filter.

#### **1. Create and print the [Collection Category](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_category) *value* filter**
The index is 8 for the Collection Category. 

In [39]:
datasetfilters_otc2l2[8]

{'id': '5f6a6fb2137a3c00',
 'legacyFieldId': None,
 'dictionaryLink': 'https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_category',
 'fieldConfig': {'type': 'Select',
  'filters': [],
  'options': {},
  'validators': [],
  'displayListId': '2837'},
 'fieldLabel': 'Collection Category',
 'searchSql': 'COLLECTION_CATEGORY = ?',
 'valueList': {'': 'All', 'T1': 'Tier 1', 'T2': 'Tier 2', 'RT': 'Real-Time'}}

The **`valueList`** shows the querable Collection Category which are **`'All'`** for all tiers of the processed data: **`'T1'`** or **`'Tier 1'`** is the highest available quality and processing level, **`'T2'`** or **`'Tier 2'`** for data not to the level of Tier 1 (i.e. significant significant cloud cover) **`'RT'`** or **`'Real-Time'`** for data that requires additional processing to be Tier 1 and Tier 2.

In [40]:
collectioncategory_filter = {'filterType': 'value',
             'filterId': datasetfilters_otc2l2[8]['id'],
             'value' : 'T2'}
collectioncategory_filter

{'filterType': 'value', 'filterId': '5f6a6fb2137a3c00', 'value': 'T2'}

#### **2. Combine into a [**metadataAnd**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataAnd) filter parameter**

- The `'filterType'` is `'and'` since we are combining the `'between'` and `'value'` filters
- The `'childFilters'` are  the `WRSpath_filter`, `WRSrow_filter`, the `'between'` filters created above and the new `collectioncategory_filter` `'value'` filter

In [41]:
wrspathrow_category_andfilter = {'filterType': 'and',
              'childFilters':[WRSpath_filter, WRSrow_filter, collectioncategory_filter]}

wrspathrow_category_andfilter

{'filterType': 'and',
 'childFilters': [{'filterId': '5e83d14fb9436d88',
   'filterType': 'between',
   'firstValue': 29,
   'secondValue': 31},
  {'filterId': '5e83d14ff1eda1b8',
   'filterType': 'between',
   'firstValue': 29,
   'secondValue': 31},
  {'filterType': 'value', 'filterId': '5f6a6fb2137a3c00', 'value': 'T2'}]}

#### **3. Create and View WRS Path, WRS Row and Collection Category Search Payload**
We expect results to show dataset between PATHs and ROWs `29` and `31` with only Tier 2 `'T2'` level quality.

In [42]:
WRS_path_row_category_payload = create_search_payload('landsat_ot_c2_l2', wrspathrow_category_andfilter, {'start' : '2023-01-01', 'end' : '2023-11-01'})
WRS_path_row_category_payload

{'maxResults': 10,
 'datasetName': 'landsat_ot_c2_l2',
 'sceneFilter': {'metadataFilter': {'filterType': 'and',
   'childFilters': [{'filterId': '5e83d14fb9436d88',
     'filterType': 'between',
     'firstValue': 29,
     'secondValue': 31},
    {'filterId': '5e83d14ff1eda1b8',
     'filterType': 'between',
     'firstValue': 29,
     'secondValue': 31},
    {'filterType': 'value', 'filterId': '5f6a6fb2137a3c00', 'value': 'T2'}]},
  'acquisitionFilter': {'start': '2023-01-01', 'end': '2023-11-01'},
  'cloudCoverFilter': {'min': 0, 'max': 50}}}

#### **4. Run and view the [*scene-search*](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) payload results for WRS Path, WRS Row and Collection Category Search**

The results from the query show **`displayID's`** with **`PPP`** (WRS Path number), **`RRR`** (WRS Row numbers) between **`29`** and **`31`**, and **`TX`** (Collection Category) **`T2`**.(see the [**Landsat 8-9 Collection 2 Level 1 Naming Convention**](https://www.usgs.gov/faqs/what-naming-convention-landsat-collections-level-1-scenes) **`LXSS_LLLL_PPPRRR_YYYYMMDD_yyyymmdd_CC_TX`**). 

In [43]:
pd.json_normalize((sendRequest(serviceUrl + "scene-search", WRS_path_row_category_payload, apiKey))['results'])

,browse,cloudCover,entityId,displayId,orderingId,metadata,hasCustomizedMetadata,publishDate,options.bulk,options.download,...,options.secondary,selected.bulk,selected.compare,selected.order,spatialBounds.type,spatialBounds.coordinates,spatialCoverage.type,spatialCoverage.coordinates,temporalCoverage.endDate,temporalCoverage.startDate
0,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",49,LC80290302023301LGN00,LC08_L2SP_029030_20231028_20231101_02_T2,None,[],None,2023-11-01 03:33:35-05,True,True,...,False,False,False,False,Polygon,"[[[-98.54497, 42.11196], [-98.54497, 44.24219]...",Polygon,"[[[-98.54497, 42.52909], [-96.29481, 42.11196]...",2023-10-28 00:00:00,2023-10-28 00:00:00
1,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",17,LC80290312023301LGN00,LC08_L2SP_029031_20231028_20231101_02_T2,None,[],None,2023-11-01 03:33:22-05,True,True,...,False,False,False,False,Polygon,"[[[-98.98939, 40.68865], [-98.98939, 42.81543]...",Polygon,"[[[-98.98939, 41.10047], [-96.78753, 40.68865]...",2023-10-28 00:00:00,2023-10-28 00:00:00
2,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",39,LC80290302023285LGN00,LC08_L2SP_029030_20231012_20231018_02_T2,None,[],None,2023-10-17 20:35:59-05,True,True,...,False,False,False,False,Polygon,"[[[-98.5375, 42.11184], [-98.5375, 44.24217], ...",Polygon,"[[[-98.5375, 42.52901], [-96.28762, 42.11184],...",2023-10-12 00:00:00,2023-10-12 00:00:00
3,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",11,LC80290312023285LGN00,LC08_L2SP_029031_20231012_20231018_02_T2,None,[],None,2023-10-17 20:33:03-05,True,True,...,False,False,False,False,Polygon,"[[[-98.98201, 40.68854], [-98.98201, 42.81538]...",Polygon,"[[[-98.98201, 41.10039], [-96.78043, 40.68854]...",2023-10-12 00:00:00,2023-10-12 00:00:00
4,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",47,LC90290292023133LGN00,LC09_L2SP_029029_20230513_20230516_02_T2,None,[],None,2023-05-16 17:47:26-05,True,True,...,False,False,False,False,Polygon,"[[[-98.12353, 43.53487], [-98.12353, 45.66528]...",Polygon,"[[[-98.12353, 43.95398], [-95.82124, 43.53487]...",2023-05-13 00:00:00,2023-05-13 00:00:00
5,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",21,LC80300292023132LGN00,LC08_L2SP_030029_20230512_20230518_02_T2,None,[],None,2023-05-18 03:20:03-05,True,True,...,False,False,False,False,Polygon,"[[[-99.6587, 43.53356], [-99.6587, 45.66655], ...",Polygon,"[[[-99.6587, 43.95515], [-97.35649, 43.53356],...",2023-05-12 00:00:00,2023-05-12 00:00:00
6,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",0,LC80290292023077LGN00,LC08_L2SP_029029_20230318_20230324_02_T2,None,[],None,2023-03-24 14:34:41-05,True,True,...,False,False,False,False,Polygon,"[[[-98.10151, 43.53373], [-98.10151, 45.66701]...",Polygon,"[[[-98.10151, 43.95555], [-95.79984, 43.53373]...",2023-03-18 00:00:00,2023-03-18 00:00:00
7,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",10,LC90300292023076LGN00,LC09_L2SP_030029_20230317_20230320_02_T2,None,[],None,2023-03-20 08:18:25-05,True,True,...,False,False,False,False,Polygon,"[[[-99.65805, 43.53498], [-99.65805, 45.6656],...",Polygon,"[[[-99.65805, 43.95448], [-97.35539, 43.53498]...",2023-03-17 00:00:00,2023-03-17 00:00:00
8,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",1,LC90300292023044LGN01,LC09_L2SP_030029_20230213_20230310_02_T2,None,[],None,2023-03-10 06:41:40-06,True,True,...,False,False,False,False,Polygon,"[[[-99.68987, 43.53458], [-99.68987, 45.66546]...",Polygon,"[[[-99.68987, 43.95423], [-97.38782, 43.53458]...",2023-02-13 00:00:00,2023-02-13 00:00:00
9,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",0,LC90300302023044LGN01,LC09_L2SP_030030_20230213_20230310_02_T2,None,[],None,2023-03-10 06:47:33-06,True,True,...,False,False,False,False,Polygon,"[[[-100.14828, 42.11355], [-100.14828, 44.2406...",Polygon,"[[[-100.14828, 42.52738], [-97.89814, 42.11355...",2023-02-13 00:00:00,2023-02-13 00:00:00


---
# [**metadata*Or***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataOr)
Like the [**metadataAnd**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataAnd), the [**metadataOr**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataOr) is used to make combininations of the [**metadataValue**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataValue) and/or [**metadataBetween**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataBetween) filter types.

#### **1. Create and print the [Landsat Scene Identifier](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary/#landsat_scene_id) *value* filter**
The index is 2 for the Landsat Scene Identifier.

In [44]:
datasetfilters_otc2l2[2]

{'id': '5e83d14fc84c9a78',
 'legacyFieldId': 27836,
 'dictionaryLink': 'https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_scene_id',
 'fieldConfig': {'type': 'Text',
  'filters': [{'type': 'StringToUpper', 'options': []},
   {'type': 'Application\\Filter\\Like', 'options': []}],
  'options': {'size': '30'},
  'validators': [],
  'numElements': '5',
  'displayListId': None},
 'fieldLabel': 'Landsat Scene Identifier',
 'searchSql': 'ENTITY_ID like ?'}

#### **2. Fill `metadataFilter` parameter with information for querying a specific `Landsat Scene Identifier`**
- The `'filterType'` is `'value'` since the `'fieldConfig':'type'` is `Text` for the `'fieldLabel': 'Landsat Scene Identifier'`
- The `'filterId': datasetfilters_tmc2L1[11]['id']` points to the `'id'` value for querying the Landsat 8-9 Operational Land Imager and Thermal Infrared Sensor Collection 2 Level-2 (`'landsat_ot_c2_l2'`) dataset
- The `'value' : 'LC80460272023260LGN00'`, `'value' : 'LC90470272023259LGN00'` and `'value' : 'LC90470272023195LGN00'` is set to query for data within a specific Landsat 8-9 Operational Land Imager and Thermal Infrared Sensor Collection 2 Level-2  (`'landsat_ot_c2_l2'`) dataset scene
- The `'operand'` is `'like'` to find scenes within the specific Scene Identifier

In [45]:
scene_identifier_orfilter = {'filterType': 'or',
              'childFilters':[
                  {'filterType': 'value',
                 'filterId': datasetfilters_tmc2L1[11]['id'],
                 'value' : 'LC80460272023260LGN00',
                           'operand' : 'like'},
                  {'filterType': 'value',
                 'filterId': datasetfilters_tmc2L1[11]['id'],
                 'value' : 'LC90470272023259LGN00',
                           'operand' : 'like'},
                  {'filterType': 'value',
                 'filterId': datasetfilters_tmc2L1[11]['id'],
                 'value' : 'LC90470272023195LGN00',
                           'operand' : 'like'}]}
scene_identifier_orfilter

{'filterType': 'or',
 'childFilters': [{'filterType': 'value',
   'filterId': '5e83d0a0fe02620e',
   'value': 'LC80460272023260LGN00',
   'operand': 'like'},
  {'filterType': 'value',
   'filterId': '5e83d0a0fe02620e',
   'value': 'LC90470272023259LGN00',
   'operand': 'like'},
  {'filterType': 'value',
   'filterId': '5e83d0a0fe02620e',
   'value': 'LC90470272023195LGN00',
   'operand': 'like'}]}

#### **3. Create the [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) payload used for querying the `LC80460272023260LGN00`, `LC90470272023259LGN00` and `LC90470272023195LGN00` scenes**

In [46]:
ot_scene_identifier_payload = create_search_payload('landsat_ot_c2_l2', scene_identifier_orfilter, {'start' : '2023-01-01', 'end' : '2023-11-01'})
ot_scene_identifier_payload

{'maxResults': 10,
 'datasetName': 'landsat_ot_c2_l2',
 'sceneFilter': {'metadataFilter': {'filterType': 'or',
   'childFilters': [{'filterType': 'value',
     'filterId': '5e83d0a0fe02620e',
     'value': 'LC80460272023260LGN00',
     'operand': 'like'},
    {'filterType': 'value',
     'filterId': '5e83d0a0fe02620e',
     'value': 'LC90470272023259LGN00',
     'operand': 'like'},
    {'filterType': 'value',
     'filterId': '5e83d0a0fe02620e',
     'value': 'LC90470272023195LGN00',
     'operand': 'like'}]},
  'acquisitionFilter': {'start': '2023-01-01', 'end': '2023-11-01'},
  'cloudCoverFilter': {'min': 0, 'max': 50}}}

#### **4. Run [*scene-search*](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search)  and view results** 
The result shows scenes with the Scene Identifiers **`LC80460272023260LGN00`**, **`LC90470272023259LGN00`** and **`LC90470272023195LGN00`**, this is indicated by the **`entityId`** names.

In [47]:
pd.json_normalize((sendRequest(serviceUrl + "scene-search", ot_scene_identifier_payload, apiKey))['results'])

""


---
## **[Logout](https://m2m.cr.usgs.gov/api/docs/reference/#logout)**<a id="logout-api"></a>

Logout so the API Key cannot be used anymore.

<div class="alert alert-block alert-warning">
    <h4> <b>NOTE:</b> The Machine-to-Machine API key expires after 2 hours of inactivity. </h4>
</div>  

In [48]:
endpoint = "logout"  
if sendRequest(serviceUrl + endpoint, None, apiKey) == None:        
    print("\n\nLogged Out\n")
else:
    print("\n\nLogout Failed\n")



Logged Out



---
<div class="alert alert-block alert-info">
    <h1> Contact Information </h1>
    <h3> Material written by Tonian Robinson<sup>1</sup> </h3>
    <ul>
        <b>Contact:</b> custserv@usgs.gov <br> 
        <b>Voice:</b> +1-605-594-6151 <br>
        <b>Organization:</b> USGS EROS User Services <br>
        <b>Date last modified:</b> 22-Aug-2024 <br>
    </ul>
    
<sup>1</sup>Earth Space Technology Services LLC., contractor to the U.S. Geological Survey, Earth Resources Observation and Science (EROS) Center, Sioux Falls, South Dakota, 57198-001, USA. Work performed under USGS contract G0121D0001.
</div>